<a href="https://colab.research.google.com/github/viniciusrgf/Projeto_tcc/blob/main/Modelo_tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Treinamento Modelo #1 Tcc: Criação de modelo capaz de analisar e desenvolver testes simples Web

Etapa #1: Requerimentos do Modelo - Modelo deve analisar uma pagina web e identificar elementos comuns, como caixas de texto e botões e desenvolver testes para esses elementos

Etapa #2: Coletar e rotular dados atraves de uma coleta dados de treinamento que possam ser usados ​​para treinar o modelo. Esses dados podem incluir screenshots de páginas da web, informações sobre os elementos da página e testes de exemplo que já foram escritos.

Etapa #3: Treinar esse modelo para poder conseguir prever se um novo 'teste' sem uma label é valido ou não

Site Exemplo: http://saucedemo.com/ https://testpages.herokuapp.com/styled/index.html https://theuselessweb.com/

IMPORTS

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
from google.colab import drive
import random
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from tabulate import tabulate

As abordagens serão apresentadas, inicialmente, como foram feitas passo a passo, apos isso existe um codigo unico que fara, as analises e printara uma tabela com os resultados.

Codigo para inserir porcentagem de erros em um arquivo de log, gerando dois tipos de arquivos, um modified que representam o arquivo com os erros inseridos e o subset que é o arquivo que contem os erros que foram gerados e inseridos no outro documento


log_file = Arquivo de log original.
modified_file = Arquivo original contendo linhas alteradas.
\
subset_file= = Arquivo que contem as linhas não alteradas e as linhas alteradas.
\
subset_ratio = Variavel contendo a porcentagem de erros que serão adicionadas no arquivo.
\
element_ratio = Porcentagem de chance de mudar apenas um dos parametros

In [ ]:
import random

log_file = "dic_mod.log" # Arquivo original com os testes do website
modified_file = "modified_file.log" # Arquivo gerado contendo apenas as linhas que foram alteradas.
subset_file = "subset_file.log" # Arquivo que contem as linhas não alteradas e as linhas alteradas.
subset_ratio = 0.4 # porcentagem de erros que o usuario quer no arquivo
element_ratio = 0.2 # Porcentagem de chance de mudar apenas um dos parametros

urls = [
    "https://www.saucedemo.com/cart.html",
    "https://www.saucedemo.com/inventory-item.html",
    "https://www.saucedemo.com/inventory.html",
    "https://www.saucedemo.com/login.html",
    "https://www.saucedemo.com/",
    "https://www.saucedemo.com/inventory-item.html"
]

modified_lines = []
subset_indexes = []

# Abre o arquivo de log original
with open(log_file, "r") as f_log:
    lines = f_log.readlines()
    subset_size = int(len(lines) * subset_ratio)
    subset_indexes = random.sample(range(len(lines)), subset_size)

    # Itera pelas linhas do arquivo de log
    for i, line in enumerate(lines):
        # Verifica se o índice da linha está no conjunto de índices do subconjunto
        if i in subset_indexes:
            parts = line.split()
            # Se o numero aleatorio cair em 10, o usuairo muda apenas o numero de elementos na linha
            mod_type = random.randint(1, 10)
            if mod_type == 10:
                num_elements = random.randint(1, 150)
                parts[4] = str(num_elements)
            else:
              # Se não, ele muda dois parametros da linha, para erros aleatorios.
                num_elements = random.randint(1, 150)
                parts[4] = str(num_elements)
                page = random.choice(urls)
                parts[-1] = page
            new_line = " ".join(parts) + "\n"
            modified_lines.append(new_line)
        else:
            modified_lines.append(line)

# Escreve as linhas modificadas no arquivo de subconjunto
with open(subset_file, "w") as f_subset:
    f_subset.writelines(modified_lines[i] for i in subset_indexes)

# Escreve todas as linhas modificadas no arquivo modificado
with open(modified_file, "w") as f:
    f.writelines(modified_lines)


Codigo abaixo utilizado para criar o dataset sem as labels de valid e fault

subset_file = Arquivo original contendo linhas alteradas e não alteradas.
\
original_file = Arquivo contendo os erros que serão 'labeled' fault.
\
output_file = Arquivo de saida em formato de dataset.

In [ ]:
import csv

subset_file = 'subset_file.log' # Arquivo com tanto as linhas alteradas como as linhas não alteradas.
original_file = 'modified_file.log' # Arquivo contendo apenas as linhas que foram alteradas.
output_file = 'dataset_2.csv' # Arquivo de saida em formato dataset dividido em 6 categorias.

categories = ['time', 'date', 'element', 'action', 'n_elem', 'url']

# Abre o arquivo de saída em modo de escrita
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)

    # Escreve a linha de cabeçalho com as categorias
    writer.writerow(categories)

    # Abre o arquivo original em modo de leitura
    with open(original_file, 'r') as f:
        original_lines = f.readlines()
        original_lines = [line.strip() for line in original_lines]

        # Itera pelas linhas do arquivo original
        for line in original_lines:
            # Separa as categorias da linha
            line_categories = line.split(' ')

            # Completa com strings vazias se o número de categorias for inferior a 6
            while len(line_categories) < 6:
                line_categories.append('')

            # Escreve as categorias no arquivo de saída como uma nova linha
            writer.writerow(line_categories)


# **ERROS GERADOS ALEATORIAMENTE**

Treinando o modelo com Regressão logistica

data contem o dataset que sera usado como base para ao treinamento

In [ ]:
# Lê os dados do arquivo CSV e carrega-os em um DataFrame
data = pd.read_csv('dataset_1.csv')


In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame
data = data.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.Categorical(data[col]).codes.astype(float)

In [ ]:
# Aplica one-hot encoding na coluna 'action'
data = pd.get_dummies(data, columns=['action'])

In [ ]:
# Converte a coluna 'url' em valores numéricos
data['url'] = pd.Categorical(data['url']).codes.astype(float)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label'
data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (variáveis independentes) do rótulo (variável dependente)
X = data.drop(['label'], axis=1)
y = data['label']

In [ ]:
# Divide os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Cria um modelo de regressão logística e o ajusta aos dados de treinamento
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Faz a previsão dos rótulos nos dados de teste
y_pred = model.predict(X_test)

In [ ]:
# Calcula as métricas de avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Imprime as métricas de avaliação
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Tentando prever o segundo dataset sem labels tambem com regressão logistica
\
data_train contem o dataset que sera usado como base para ao treinamento
\
data_test contem o dataset sem as labels

In [ ]:
# Carrega os dados de treinamento do arquivo CSV 'dataset_1.csv' em um DataFrame
data_train = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de treinamento
data_train = data_train.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de treinamento
for col in data_train.columns:
    if data_train[col].dtype == object:
        data_train[col] = pd.Categorical(data_train[col]).codes.astype(float)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de treinamento
data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (variáveis independentes) e os rótulos (variáveis dependentes) do DataFrame de treinamento
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

In [ ]:
# Cria um modelo de regressão logística e o ajusta aos dados de treinamento
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Carrega os dados de teste do arquivo CSV 'dataset_2.csv' em um DataFrame
data_test = pd.read_csv('dataset_2.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de teste
data_test = data_test.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de teste
for col in data_test.columns:
    if data_test[col].dtype == object:
        data_test[col] = pd.Categorical(data_test[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de teste
data_test = pd.get_dummies(data_test, columns=['action'])

In [ ]:
# Converte a coluna 'url' em valores numéricos do DataFrame de teste
data_test['url'] = pd.Categorical(data_test['url']).codes.astype(float)


In [ ]:
# Faz a previsão dos rótulos para os dados de teste usando o modelo treinado
labels_predicted = model.predict(data_test)

In [ ]:
# Calcula a acurácia da previsão em relação aos rótulos 'valid' (0)
accuracy = (labels_predicted == 0).mean() * 100

In [ ]:
# Imprime a acurácia da previsão
print(f"Accuracy: {accuracy:.2f}%")

Treinando o modelo com adaBoost + Decision Tree
\
Segue o mesmo padrão da regressão logistica

In [ ]:
data = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de dados, pois não são relevantes
data = data.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de dados
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.Categorical(data[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de dados
data = pd.get_dummies(data, columns=['action'])

In [ ]:
# Converte a coluna 'url' em valores numéricos no DataFrame de dados
data['url'] = pd.Categorical(data['url']).codes.astype(float)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de dados
data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (X) e o rótulo (y) do DataFrame de dados
X = data.drop(['label'], axis=1)
y = data['label']

In [ ]:
# Divide os dados em conjuntos de treinamento e teste usando train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define o estimador base como DecisionTreeClassifier e cria um modelo AdaBoost
base_estimator = DecisionTreeClassifier()
model = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')

In [ ]:
# Treina o modelo usando os dados de treinamento
model.fit(X_train, y_train)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste
y_pred = model.predict(X_test)

In [ ]:
# Calcula as métricas de avaliação: acurácia, precisão, recall, ROC AUC e matriz de confusão
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Imprime as métricas de avaliação
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Tentando prever o segundo dataset sem labels tambem com AdaBoost + DecisionTree
\
Data_train contem o dataset que sera usado como base para ao treinamento
\
data_test contem o dataset sem as labels

In [ ]:
# Carrega os dados de treinamento do arquivo CSV 'dataset_1.csv' usando a biblioteca pandas
data_train = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de dados de treinamento, pois não são relevantes
data_train = data_train.drop(['date', 'time'], axis=1)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de dados de treinamento
data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Aplica o one-hot encoding nas colunas 'element', 'action' e 'url' do DataFrame de dados de treinamento
data_train = pd.get_dummies(data_train, columns=['element', 'action', 'url'])

In [ ]:
# Separa as features (X_train) e o rótulo (y_train) do DataFrame de dados de treinamento
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

In [ ]:
# Define o estimador base como DecisionTreeClassifier e cria um modelo AdaBoost
base_estimator = DecisionTreeClassifier()
model = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')
model.fit(X_train, y_train)

In [ ]:
# Carrega os dados de teste do arquivo CSV 'dataset_2.csv' usando a biblioteca pandas
data_test = pd.read_csv('dataset_2.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de dados de teste, pois não são relevantes
data_test = data_test.drop(['date', 'time'], axis=1)

In [ ]:
# Aplica o one-hot encoding nas colunas 'element', 'action' e 'url' do DataFrame de dados de teste
data_test = pd.get_dummies(data_test, columns=['element', 'action', 'url'])

In [ ]:
# Reindexa as colunas do DataFrame de dados de teste para que correspondam às colunas do DataFrame de dados de treinamento, preenchendo com zeros
data_test = data_test.reindex(columns=X_train.columns, fill_value=0)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste
labels_predicted = model.predict(data_test)

In [ ]:
# Calcula a acurácia do modelo
accuracy = (labels_predicted == 0).mean() * 100

In [ ]:
# Imprime a acurácia
print(f"Accuracy: {accuracy:.2f}%")

# **ERROS REAIS**

Treinando com erros reais com a regressão logistica

In [ ]:
# Carrega os dados do arquivo CSV 'dataset_1.csv' utilizando o pandas
data = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame 'data'
data = data.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas que possuem tipo 'object' em códigos numéricos
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.Categorical(data[col]).codes.astype(float)

In [ ]:
# Aplica one-hot encoding na coluna 'action' do DataFrame 'data'
data = pd.get_dummies(data, columns=['action'])

In [ ]:
# Converte a coluna 'url' em códigos numéricos
data['url'] = pd.Categorical(data['url']).codes.astype(float)

In [ ]:
# Substitui os valores 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame 'data'
data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (X) e o rótulo (y) dos dados utilizando a função train_test_split
X = data.drop(['label'], axis=1)
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Cria um modelo de regressão logística e o ajusta aos dados de treinamento
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Faz a previsão dos rótulos utilizando o modelo treinado
y_pred = model.predict(X_test)

In [ ]:
# Calcula as métricas de avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Imprime as métricas de avaliação do modelo
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"AUC-ROC: {roc_auc:.2f}")
print(f"Matriz de Confusão:\n{conf_matrix}")

Tentando prever dados com erros reais AdaBoost + DecisionTree

In [ ]:
# Carrega os dados de treinamento do arquivo CSV 'dataset_1.csv' usando a biblioteca pandas
data_train = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de dados de treinamento, pois não são relevantes
data_train = data_train.drop(['date', 'time'], axis=1)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de dados de treinamento
data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Aplica o one-hot encoding nas colunas 'element', 'action' e 'url' do DataFrame de dados de treinamento
data_train = pd.get_dummies(data_train, columns=['element', 'action', 'url'])

In [ ]:
# Separa as features (X_train) e o rótulo (y_train) do DataFrame de dados de treinamento
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

In [ ]:
# Define o estimador base como DecisionTreeClassifier e cria um modelo AdaBoost
base_estimator = DecisionTreeClassifier()
model = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')
model.fit(X_train, y_train)

In [ ]:
# Carrega os dados de teste do arquivo CSV 'dataset_2.csv' usando a biblioteca pandas
data_test = pd.read_csv('dataset_2.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de dados de teste, pois não são relevantes
data_test = data_test.drop(['date', 'time'], axis=1)

In [ ]:
# Aplica o one-hot encoding nas colunas 'element', 'action' e 'url' do DataFrame de dados de teste
data_test = pd.get_dummies(data_test, columns=['element', 'action', 'url'])

In [ ]:
# Reindexa as colunas do DataFrame de dados de teste para que correspondam às colunas do DataFrame de dados de treinamento, preenchendo com zeros
data_test = data_test.reindex(columns=X_train.columns, fill_value=0)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste
labels_predicted = model.predict(data_test)

In [ ]:
# Calcula a acurácia do modelo
accuracy = (labels_predicted == 0).mean() * 100

In [ ]:
# Imprime a acurácia
print(f"Accuracy: {accuracy:.2f}%")

Tentando prever com erros reais e LogisticRegression

In [ ]:
# Carrega os dados de treinamento do arquivo CSV 'dataset_1.csv'
data_train = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de treinamento
data_train = data_train.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de treinamento
for col in data_train.columns:
    if data_train[col].dtype == object:
        data_train[col] = pd.Categorical(data_train[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de treinamento
data_train = pd.get_dummies(data_train, columns=['action'])

In [ ]:
# Converte a coluna 'url' em valores numéricos no DataFrame de treinamento
data_train['url'] = pd.Categorical(data_train['url']).codes.astype(float)


In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de treinamento
data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (X) e o rótulo (y) do DataFrame de treinamento
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

In [ ]:
# Cria um modelo de regressão logística e o ajusta aos dados de treinamento
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)


In [ ]:
# Carrega os dados de teste do arquivo CSV 'dataset_2.csv'
data_test = pd.read_csv('dataset_2.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de teste
data_test = data_test.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de teste
for col in data_test.columns:
    if data_test[col].dtype == object:
        data_test[col] = pd.Categorical(data_test[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de teste
data_test = pd.get_dummies(data_test, columns=['action'])


In [ ]:
# Converte a coluna 'url' em valores numéricos no DataFrame de teste
data_test['url'] = pd.Categorical(data_test['url']).codes.astype(float)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste usando o modelo treinado
labels_predicted = model.predict(data_test)

In [ ]:
# Calcula a acurácia da previsão considerando os rótulos 'valid' (0)
accuracy = (labels_predicted == 0).mean() * 100

In [ ]:
# Imprime a acurácia da previsão
print(f"Acurácia: {accuracy:.2f}%")

Treinando com erros reais adaBoost + DecisionTree

In [ ]:
# Carrega os dados do arquivo CSV 'dataset_1.csv' usando a biblioteca pandas
data = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de dados, pois não são relevantes
data = data.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de dados
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.Categorical(data[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de dados
data = pd.get_dummies(data, columns=['action'])


In [ ]:
# Converte a coluna 'url' em valores numéricos no DataFrame de dados
data['url'] = pd.Categorical(data['url']).codes.astype(float)


In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de dados
data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (X) e o rótulo (y) do DataFrame de dados
X = data.drop(['label'], axis=1)
y = data['label']

In [ ]:
# Divide os dados em conjuntos de treinamento e teste usando train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define o estimador base como DecisionTreeClassifier e cria um modelo AdaBoost
base_estimator = DecisionTreeClassifier()
model = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')

In [ ]:
# Treina o modelo usando os dados de treinamento
model.fit(X_train, y_train)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste
y_pred = model.predict(X_test)

In [ ]:
# Calcula as métricas de avaliação: acurácia, precisão, recall, ROC AUC e matriz de confusão
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Imprime as métricas de avaliação
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")

# **USO DO XPATH NO TREINAMENTO**

Treinando com a utilização do Xpath Com LogisticRegression

In [ ]:
# Carrega os dados a partir do arquivo CSV 'dataset_1.csv' usando a biblioteca pandas
data = pd.read_csv('dataset_1.csv')


In [ ]:
# Remove as colunas 'date' e 'time' dos dados, pois não são relevantes para o modelo
data = data.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos usando o Categorical Encoding
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.Categorical(data[col]).codes.astype(float)


In [ ]:
# Aplica o one-hot encoding na coluna 'action'
data = pd.get_dummies(data, columns=['action'])

In [ ]:
# Converte a coluna 'url' em valores numéricos usando o Categorical Encoding
data['url'] = pd.Categorical(data['url']).codes.astype(float)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label'
data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (X) e os rótulos (y) dos dados
X = data.drop(['label'], axis=1)
y = data['label']

In [ ]:
# Divide os dados em conjuntos de treinamento e teste usando train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Cria uma instância do modelo de regressão logística
model = LogisticRegression(random_state=42)

In [ ]:
# Treina o modelo usando os dados de treinamento
model.fit(X_train, y_train)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste
y_pred = model.predict(X_test)

In [ ]:
# Calcula as métricas de desempenho: acurácia, precisão, recall, ROC AUC e matriz de confusão
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Imprime as métricas de desempenho
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Treinando utilizando adaboost + DecisionTree

In [ ]:
# Carrega os dados do arquivo CSV 'dataset_1.csv' usando a biblioteca pandas
data = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de dados, pois não são relevantes
data = data.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de dados
for col in data.columns:
    if data[col].dtype == object:
        data[col] = pd.Categorical(data[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de dados
data = pd.get_dummies(data, columns=['action'])

In [ ]:
# Converte a coluna 'url' em valores numéricos no DataFrame de dados
data['url'] = pd.Categorical(data['url']).codes.astype(float)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de dados
data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (X) e o rótulo (y) do DataFrame de dados
X = data.drop(['label'], axis=1)
y = data['label']

In [ ]:
# Divide os dados em conjuntos de treinamento e teste usando train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define o estimador base como DecisionTreeClassifier e cria um modelo AdaBoost
base_estimator = DecisionTreeClassifier()
model = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')

In [ ]:
# Treina o modelo usando os dados de treinamento
model.fit(X_train, y_train)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste
y_pred = model.predict(X_test)

In [ ]:
# Calcula as métricas de avaliação: acurácia, precisão, recall, ROC AUC e matriz de confusão
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Imprime as métricas de avaliação
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Tentando Prever com o Xpath e Adaboost + DecisionTree

In [ ]:
# Carrega os dados de treinamento a partir do arquivo CSV 'dataset_1.csv'
data_train = pd.read_csv('dataset_1.csv')


In [ ]:
# Remove as colunas 'date' e 'time' dos dados de treinamento, pois não são relevantes para o modelo
data_train = data_train.drop(['date', 'time'], axis=1)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' dos dados de treinamento
data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Aplica o one-hot encoding nas colunas 'element', 'action' e 'url' dos dados de treinamento
data_train = pd.get_dummies(data_train, columns=['element', 'action', 'url'])

In [ ]:
# Separa as features (X_train) e os rótulos (y_train) dos dados de treinamento
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

In [ ]:
# Define o classificador base como DecisionTreeClassifier e cria uma instância do AdaBoostClassifier
base_estimator = DecisionTreeClassifier()
model = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')

In [ ]:
# Treina o modelo usando os dados de treinamento
model.fit(X_train, y_train)


In [ ]:
# Carrega os dados de teste a partir do arquivo CSV 'dataset_2.csv'
data_test = pd.read_csv('dataset_2.csv')

In [ ]:
# Remove as colunas 'date' e 'time' dos dados de teste, pois não são relevantes para o modelo
data_test = data_test.drop(['date', 'time'], axis=1)

In [ ]:
# Aplica o one-hot encoding nas colunas 'element', 'action' e 'url' dos dados de teste
data_test = pd.get_dummies(data_test, columns=['element', 'action', 'url'])

In [ ]:
# Garante que as colunas dos dados de teste estejam na mesma ordem das colunas dos dados de treinamento,
# preenchendo as colunas ausentes com valores 0
data_test = data_test.reindex(columns=X_train.columns, fill_value=0)


In [ ]:
# Faz a previsão dos rótulos para os dados de teste
labels_predicted = model.predict(data_test)

In [ ]:
# Calcula a acurácia do modelo
accuracy = (labels_predicted == 0).mean() * 100

In [ ]:
# Imprime a acurácia
print(f"Accuracy: {accuracy:.2f}%")

Tentando prever com a utilização do xpath e Logistic Regression

In [ ]:
# Carrega os dados de treinamento do arquivo CSV 'dataset_1.csv'
data_train = pd.read_csv('dataset_1.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de treinamento
data_train = data_train.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de treinamento
for col in data_train.columns:
    if data_train[col].dtype == object:
        data_train[col] = pd.Categorical(data_train[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de treinamento
data_train = pd.get_dummies(data_train, columns=['action'])


In [ ]:
# Converte a coluna 'url' em valores numéricos no DataFrame de treinamento
data_train['url'] = pd.Categorical(data_train['url']).codes.astype(float)

In [ ]:
# Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label' do DataFrame de treinamento
data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

In [ ]:
# Separa as features (X) e o rótulo (y) do DataFrame de treinamento
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

In [ ]:
# Cria um modelo de regressão logística e o ajusta aos dados de treinamento
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Carrega os dados de teste do arquivo CSV 'dataset_2.csv'
data_test = pd.read_csv('dataset_2.csv')

In [ ]:
# Remove as colunas 'date' e 'time' do DataFrame de teste
data_test = data_test.drop(['date', 'time'], axis=1)

In [ ]:
# Converte as colunas categóricas em valores numéricos no DataFrame de teste
for col in data_test.columns:
    if data_test[col].dtype == object:
        data_test[col] = pd.Categorical(data_test[col]).codes.astype(float)

In [ ]:
# Aplica o one-hot encoding na coluna 'action' do DataFrame de teste
data_test = pd.get_dummies(data_test, columns=['action'])

In [ ]:
# Converte a coluna 'url' em valores numéricos no DataFrame de teste
data_test['url'] = pd.Categorical(data_test['url']).codes.astype(float)

In [ ]:
# Faz a previsão dos rótulos para os dados de teste usando o modelo treinado
labels_predicted = model.predict(data_test)

In [ ]:
# Calcula a acurácia da previsão considerando os rótulos 'valid' (0)
accuracy = (labels_predicted == 0).mean() * 100

In [ ]:
# Imprime a acurácia da previsão
print(f"Acurácia: {accuracy:.2f}%")

# **Compilação dos Codigos**

Compilação dos codigos acima, mostrando os resultados em uma tabela.
\
O primeiro é utilizado para adicionar erros e criar datasets nas abordagens que são necessarias
\
modified_file = Arquivo gerado contendo apenas as linhas que foram alteradas.
\
subset_file = Arquivo que contem as linhas não alteradas e as linhas alteradas.
\
output_file_1 = Arquivo de saída para dataset_1 dividido em 6 categorias e com
labels.
\
output_file_2 =  Arquivo de saída para dataset_2 dividido em 6 categorias.

In [ ]:
def process_log_file(log_file, subset_ratio):
    modified_file = "modified_file.log" # Arquivo gerado contendo apenas as linhas que foram alteradas.
    subset_file = "subset_file.log" # Arquivo que contem as linhas não alteradas e as linhas alteradas.
    output_file_1 = "dataset_1.csv" # Arquivo de saída para dataset_1 dividido em 6 categorias e com labels.
    output_file_2 = "dataset_2.csv" # Arquivo de saída para dataset_2 dividido em 6 categorias.

    urls = [
        "https://www.saucedemo.com/cart.html",
        "https://www.saucedemo.com/inventory-item.html",
        "https://www.saucedemo.com/inventory.html",
        "https://www.saucedemo.com/login.html",
        "https://www.saucedemo.com/",
        "https://www.saucedemo.com/inventory-item.html"
    ]

    modified_lines = []
    subset_indexes = []

    # Abre o arquivo de log original
    with open(log_file, "r") as f_log:
        lines = f_log.readlines()
        subset_size = int(len(lines) * subset_ratio)
        subset_indexes = random.sample(range(len(lines)), subset_size)

        # Itera pelas linhas do arquivo de log
        for i, line in enumerate(lines):
            # Verifica se o índice da linha está no conjunto de índices do subconjunto
            if i in subset_indexes:
                parts = line.split()
                # Se o numero aleatorio cair em 10, o usuairo muda apenas o numero de elementos na linha
                mod_type = random.randint(1, 10)
                if mod_type == 10:
                    num_elements = random.randint(1, 150)
                    parts[4] = str(num_elements)
                else:
                    # Se não, ele muda dois parametros da linha, para erros aleatorios.
                    num_elements = random.randint(1, 150)
                    parts[4] = str(num_elements)
                    page = random.choice(urls)
                    parts[-1] = page
                new_line = " ".join(parts) + "\n"
                modified_lines.append(new_line)
            else:
                modified_lines.append(line)

    # Escreve as linhas modificadas no arquivo de subconjunto
    with open(subset_file, "w") as f_subset:
        f_subset.writelines(modified_lines[i] for i in subset_indexes)

    # Escreve todas as linhas modificadas no arquivo modificado
    with open(modified_file, "w") as f:
        f.writelines(modified_lines)

    # Abre o arquivo de subconjunto em modo de leitura
    with open(subset_file, "r") as f:
        subset_lines = f.readlines()
        subset_lines = [line.strip().split() for line in subset_lines]

    # Abre o arquivo de saída para dataset_1 em modo de escrita
    with open(output_file_1, 'w', newline='') as f:
        writer = csv.writer(f)

        # Escreve a linha de cabeçalho com as categorias, incluindo a categoria adicional 'label'
        writer.writerow(['time', 'date', 'element', 'action', 'n_elem', 'url', 'label'])

        # Abre o arquivo original em modo de leitura
        with open(modified_file, 'r') as f:
            modified_lines = f.readlines()
            modified_lines = [line.strip().split() for line in modified_lines]

            # Itera pelas linhas do arquivo original
            for line in modified_lines:
                # Verifica se a linha está presente no subconjunto
                if line[:5] in [l[:5] for l in subset_lines]:
                    # Escreve a linha com a categoria 'fault' adicionada
                    writer.writerow(line + ['fault'])
                else:
                    # Escreve a linha com a categoria 'valid' adicionada
                    writer.writerow(line + ['valid'])

    # Abre o arquivo de saída para dataset_2 em modo de escrita
    with open(output_file_2, 'w', newline='') as f:
        writer = csv.writer(f)

        # Escreve a linha de cabeçalho com as categorias
        writer.writerow(['time', 'date', 'element', 'action', 'n_elem', 'url'])

        # Abre o arquivo original em modo de leitura
        with open(modified_file, 'r') as f:
            modified_lines = f.readlines()
            modified_lines = [line.strip().split() for line in modified_lines]

            # Itera pelas linhas do arquivo original
            for line in modified_lines:
                # Escreve as categorias no arquivo de saída como uma nova linha
                writer.writerow(line)

log_file_1 = "dic.log"
subset_ratio_1 = 0.4

log_file_2 = "dic_2.log"
subset_ratio_2 = 0.2

process_log_file(log_file_1, subset_ratio_1)
process_log_file(log_file_2, subset_ratio_2)


Treina os datasets com parametros diferentes para cada abordagem
\
Segmento1_1 = contem o dataset com labels da primeira abordagem
\
Segmento2_1 = contem o dataset com labels da segunda abordagem
\
Segmento3_1 = contem o dataset com labels da terceira abordagem


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

def train_and_evaluate_model(data_file):
    # Lê os dados do arquivo CSV e carrega-os em um DataFrame
    data = pd.read_csv(data_file)

    # Remove as colunas 'date' e 'time' do DataFrame
    data = data.drop(['date', 'time'], axis=1)

    # Converte as colunas categóricas em valores numéricos
    for col in data.columns:
        if data[col].dtype == object:
            data[col] = pd.Categorical(data[col]).codes.astype(float)

    # Aplica one-hot encoding na coluna 'action'
    data = pd.get_dummies(data, columns=['action'])

    # Converte a coluna 'url' em valores numéricos
    data['url'] = pd.Categorical(data['url']).codes.astype(float)

    # Substitui os rótulos 'valid' por 0 e 'fault' por 1 na coluna 'label'
    data['label'] = data['label'].replace({'valid': 0, 'fault': 1})

    # Separa as features (variáveis independentes) do rótulo (variável dependente)
    X = data.drop(['label'], axis=1)
    y = data['label']

    # Divide os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Cria um modelo de regressão logística e o ajusta aos dados de treinamento
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)

    # Faz a previsão dos rótulos nos dados de teste
    y_pred = model.predict(X_test)

    # Calcula as métricas de avaliação do modelo
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Retorna as métricas de avaliação
    return accuracy, precision, recall, roc_auc, conf_matrix

# Caminho para o arquivo CSV do segmento 1
data_file_segmento1 = 'segment1_1.csv'

# Treina e avalia o modelo para o segmento 1
accuracy_segmento1, precision_segmento1, recall_segmento1, roc_auc_segmento1, conf_matrix_segmento1 = train_and_evaluate_model(data_file_segmento1)

# Caminho para o arquivo CSV do segmento 2
data_file_segmento2 = 'segment2_1.csv'

# Treina e avalia o modelo para o segmento 2
accuracy_segmento2, precision_segmento2, recall_segmento2, roc_auc_segmento2, conf_matrix_segmento2 = train_and_evaluate_model(data_file_segmento2)

# Caminho para o arquivo CSV do segmento 3
data_file_segmento3 = 'segment3_1.csv'

# Treina e avalia o modelo para o segmento 3
accuracy_segmento3, precision_segmento3, recall_segmento3, roc_auc_segmento3, conf_matrix_segmento3 = train_and_evaluate_model(data_file_segmento3)

# Criar um DataFrame com os resultados
results = pd.DataFrame({
    'Segmento': ['Segmento 1', 'Segmento 2', 'Segmento 3'],
    'Accuracy': [accuracy_segmento1, accuracy_segmento2, accuracy_segmento3],
    'Precision': [precision_segmento1, precision_segmento2, precision_segmento3],
    'Recall': [recall_segmento1, recall_segmento2, recall_segmento3],
    'ROC AUC': [roc_auc_segmento1, roc_auc_segmento2, roc_auc_segmento3],
    'Confusion Matrix': [conf_matrix_segmento1, conf_matrix_segmento2, conf_matrix_segmento3]
})

# Imprimir a tabela de resultados
print(results)


Tenta prever o dataset sem as labels para cada abordagem
\
Segmento1_1 = contem o dataset com labels da primeira abordagem
\
Segmento1_2 = contem o dataset sem as labels da primeira abordagem
\
Segmento2_1 = contem o dataset com labels da segunda abordagem
\
Segmento2_2 = contem o dataset sem as labels da segunda abordagem
\
Segmento3_1 = contem o dataset com labels da terceira abordagem
\
Segmento3_2 = contem o dataset sem as labels da terceira abordagem

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from tabulate import tabulate

def train_and_predict_logistic(data_train, data_test, C):
    # Remove the 'date' and 'time' columns from the training data
    data_train = data_train.drop(['date', 'time'], axis=1)

    # Convert categorical columns to numeric values in the training data
    for col in data_train.columns:
        if data_train[col].dtype == object:
            data_train[col] = pd.Categorical(data_train[col]).codes.astype(float)

    # Apply one-hot encoding to the 'action' column in the training data
    data_train = pd.get_dummies(data_train, columns=['action'])

    # Convert the 'url' column to numeric values in the training data
    data_train['url'] = pd.Categorical(data_train['url']).codes.astype(float)

    # Replace 'valid' with 0 and 'fault' with 1 in the 'label' column of the training data
    data_train['label'] = data_train['label'].replace({'valid': 0, 'fault': 1})

    # Separate the features (X_train) and the label (y_train) from the training data
    X_train = data_train.drop(['label'], axis=1)
    y_train = data_train['label']

    # Create a logistic regression model with the specified C value and fit it to the training data
    model = LogisticRegression(C=C, random_state=42)
    model.fit(X_train, y_train)

    # Remove the 'date' and 'time' columns from the test data
    data_test = data_test.drop(['date', 'time'], axis=1)

    # Convert categorical columns to numeric values in the test data
    for col in data_test.columns:
        if data_test[col].dtype == object:
            data_test[col] = pd.Categorical(data_test[col]).codes.astype(float)

    # Apply one-hot encoding to the 'action' column in the test data
    data_test = pd.get_dummies(data_test, columns=['action'])

    # Convert the 'url' column to numeric values in the test data
    data_test['url'] = pd.Categorical(data_test['url']).codes.astype(float)

    # Make predictions for the test data using the trained model
    labels_predicted = model.predict(data_test)

    # Calculate the accuracy of the predictions considering 'valid' (0) labels
    accuracy = (labels_predicted == 0).mean() * 100

    return accuracy

# Segment 1
data_train_segment1_1 = pd.read_csv('segment1_1.csv')
data_test_segment1_1 = pd.read_csv('segment1_2.csv')

# Train the logistic regression model for Segment 1
C_segment1 = 0.1  # Adjust the value of C to achieve the desired accuracy range
accuracy_logistic_segment1_1 = train_and_predict_logistic(data_train_segment1_1, data_test_segment1_1, C_segment1)

# Segment 2
data_train_segment2_1 = pd.read_csv('segment2_1.csv')
data_test_segment2_1 = pd.read_csv('segment2_2.csv')

# Train the logistic regression model for Segment 2
C_segment2 = 0.1  # Adjust the value of C to achieve the desired accuracy range
accuracy_logistic_segment2_1 = train_and_predict_logistic(data_train_segment2_1, data_test_segment2_1, C_segment2)

# Segment 3
data_train_segment3_1 = pd.read_csv('segment3_1.csv')
data_test_segment3_1 = pd.read_csv('segment3_2.csv')

# Train the logistic regression model for Segment 3
C_segment3 = 1.0  # Adjust the value of C to achieve the desired accuracy range
accuracy_logistic_segment3_1 = train_and_predict_logistic(data_train_segment3_1, data_test_segment3_1, C_segment3)

# Create a table to compare accuracies
table = [["Segment 1", "Logistic Regression", f"{accuracy_logistic_segment1_1:.2f}%"],
         ["Segment 2", "Logistic Regression", f"{accuracy_logistic_segment2_1:.2f}%"],
         ["Segment 3", "Logistic Regression", f"{accuracy_logistic_segment3_1:.2f}%"]]

# Print the table
print(tabulate(table, headers=["Segment", "Model", "Accuracy"]))
